In [ ]:
!pip install hdbscan
!pip install -U scikit-learn
!pip install matplotlib

In [ ]:
import time
import warnings
from itertools import cycle, islice

import matplotlib.pyplot as plt
import numpy as np

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
import hdbscan

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 500
seed = 30
noisy_circles = datasets.make_circles(
    n_samples=n_samples, factor=0.5, noise=0.05, random_state=seed
)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=seed)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=seed)
rng = np.random.RandomState(seed)
no_structure = rng.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state
)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 13))
plt.subplots_adjust(
    left=0.02, right=0.98, bottom=0.001, top=0.95, wspace=0.05, hspace=0.01
)

plot_num = 1

default_base = {
    "quantile": 0.3,
    "eps": 0.3,
    "damping": 0.9,
    "preference": -200,
    "n_neighbors": 3,
    "n_clusters": 3,
    "min_samples": 7,
    "xi": 0.05,
    "min_cluster_size": 0.1,
    "allow_single_cluster": True,
    "hdbscan_min_cluster_size": 15,
    "hdbscan_min_samples": 3,
    "random_state": 42,
}

datasets = [
    (
        noisy_circles,
        {
            "damping": 0.77,
            "preference": -240,
            "quantile": 0.2,
            "n_clusters": 2,
            "min_samples": 7,
            "xi": 0.08,
        },
    ),
    (
        noisy_moons,
        {
            "damping": 0.75,
            "preference": -220,
            "n_clusters": 2,
            "min_samples": 7,
            "xi": 0.1,
        },
    ),
    (
        varied,
        {
            "eps": 0.18,
            "n_neighbors": 2,
            "min_samples": 7,
            "xi": 0.01,
            "min_cluster_size": 0.2,
        },
    ),
    (
        aniso,
        {
            "eps": 0.15,
            "n_neighbors": 2,
            "min_samples": 7,
            "xi": 0.1,
            "min_cluster_size": 0.2,
        },
    ),
    (blobs, {"min_samples": 7, "xi": 0.1, "min_cluster_size": 0.2}),
    (no_structure, {}),
]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params["quantile"])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params["n_neighbors"], include_self=False
    )
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(
        n_clusters=params["n_clusters"],
        random_state=params["random_state"],
    )
    ward = cluster.AgglomerativeClustering(
        n_clusters=params["n_clusters"], linkage="ward", connectivity=connectivity
    )
    spectral = cluster.SpectralClustering(
        n_clusters=params["n_clusters"],
        eigen_solver="arpack",
        affinity="nearest_neighbors",
        random_state=params["random_state"],
    )
    dbscan = cluster.DBSCAN(eps=params["eps"])
    hdbscan_clust = hdbscan.HDBSCAN(
        min_samples=params["hdbscan_min_samples"],
        min_cluster_size=params["hdbscan_min_cluster_size"],
        allow_single_cluster=params["allow_single_cluster"],
    )
    optics = cluster.OPTICS(
        min_samples=params["min_samples"],
        xi=params["xi"],
        min_cluster_size=params["min_cluster_size"],
    )
    affinity_propagation = cluster.AffinityPropagation(
        damping=params["damping"],
        preference=params["preference"],
        random_state=params["random_state"],
    )
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average",
        metric="cityblock",
        n_clusters=params["n_clusters"],
        connectivity=connectivity,
    )
    birch = cluster.Birch(n_clusters=params["n_clusters"])
    gmm = mixture.GaussianMixture(
        n_components=params["n_clusters"],
        covariance_type="full",
        random_state=params["random_state"],
    )

    clustering_algorithms = (
        ("MiniBatch\nKMeans", two_means),
        ("Affinity\nPropagation", affinity_propagation),
        ("MeanShift", ms),
        ("Spectral\nClustering", spectral),
        ("Ward", ward),
        ("Agglomerative\nClustering", average_linkage),
        ("DBSCAN", dbscan),
        ("HDBSCAN", hdbscan_clust),
        ("OPTICS", optics),
        ("BIRCH", birch),
        ("Gaussian\nMixture", gmm),
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the "
                + "connectivity matrix is [0-9]{1,2}"
                + " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning,
            )
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding"
                + " may not work as expected.",
                category=UserWarning,
            )
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, "labels_"):
            y_pred = algorithm.labels_.astype(int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(
            list(
                islice(
                    cycle(
                        [
                            "#377eb8",
                            "#ff7f00",
                            "#4daf4a",
                            "#f781bf",
                            "#a65628",
                            "#984ea3",
                            "#999999",
                            "#e41a1c",
                            "#dede00",
                        ]
                    ),
                    int(max(y_pred) + 1),
                )
            )
        )
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(
            0.99,
            0.01,
            ("%.2fs" % (t1 - t0)).lstrip("0"),
            transform=plt.gca().transAxes,
            size=15,
            horizontalalignment="right",
        )
        plot_num += 1

plt.show()


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Set the number of samples per class
n_samples_per_class = 100
n_classes = 10
seed = 42  # Seed for reproducibility

# Initialize lists to hold sampled images and labels
sampled_images = []
sampled_labels = []

# Iterate over each class and sample images
for class_label in range(n_classes):
    # Get all images and labels for the current class
    class_images = x_train[y_train == class_label]
    class_labels = y_train[y_train == class_label]

    # Set the seed for numpy's random number generator before sampling
    np.random.seed(seed + class_label)  # Each class with a unique seed

    # Randomly sample n_samples_per_class images from the current class
    sampled_indices = np.random.choice(len(class_images), n_samples_per_class, replace=False)
    sampled_images.append(class_images[sampled_indices])
    sampled_labels.append(class_labels[sampled_indices])

# Convert the lists to numpy arrays
sampled_images = np.concatenate(sampled_images, axis=0)
sampled_labels = np.concatenate(sampled_labels, axis=0)

# Shuffle the sampled dataset
np.random.seed(seed)
shuffled_indices = np.random.permutation(len(sampled_labels))
sampled_images = sampled_images[shuffled_indices]
sampled_labels = sampled_labels[shuffled_indices]

# Flatten the images for clustering
n_samples, width, height = sampled_images.shape
flattened_images = sampled_images.reshape(n_samples, width * height)

# Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
flattened_images_scaled = scaler.fit_transform(flattened_images)

# Verify the shape of the sampled data
print("Sampled images shape:", sampled_images.shape)  # Should be (1000, 28, 28)
print("Sampled labels shape:", sampled_labels.shape)  # Should be (1000,)
print("First 10 sampled labels:", sampled_labels[:10])


In [ ]:
from sklearn.cluster import AgglomerativeClustering, KMeans, SpectralClustering
from sklearn.mixture import GaussianMixture

# Define clustering algorithms
clustering_algorithms = {
    'Agglomerative Clustering': AgglomerativeClustering(n_clusters=10),
    'KMeans': KMeans(n_clusters=10, random_state=seed),
    'Gaussian Mixture': GaussianMixture(n_components=10, random_state=seed),
    'Spectral Clustering': SpectralClustering(n_clusters=10, affinity='nearest_neighbors', random_state=seed)
}

# Dictionary to hold clustering results
clustering_results = {}

# Apply each clustering algorithm
for name, algorithm in clustering_algorithms.items():
    if name == 'Gaussian Mixture':
        clusters = algorithm.fit_predict(flattened_images_scaled)
    else:
        clusters = algorithm.fit_predict(flattened_images_scaled)
    clustering_results[name] = clusters


In [ ]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

# Dictionary to hold evaluation results
evaluation_results = {}

# Compute evaluation metrics for each clustering result
for name, clusters in clustering_results.items():
    rand_index = adjusted_rand_score(sampled_labels, clusters)
    mutual_info = adjusted_mutual_info_score(sampled_labels, clusters)
    evaluation_results[name] = {
        'Rand Index': rand_index,
        'Mutual Information': mutual_info
    }

# Print evaluation results
for name, scores in evaluation_results.items():
    print(f"{name}:")
    print(f"  Rand Index: {scores['Rand Index']}")
    print(f"  Mutual Information: {scores['Mutual Information']}")


In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score

# Function to find cluster centers
def find_cluster_centers(clusters, data):
    centers = []
    for cluster_id in np.unique(clusters):
        cluster_points = data[clusters == cluster_id]
        center = cluster_points.mean(axis=0)
        centers.append(center)
    return np.array(centers)

# Perform 1-NN classification using cluster centers
for name, clusters in clustering_results.items():
    # Find cluster centers
    cluster_centers = find_cluster_centers(clusters, flattened_images_scaled)

    # Fit 1-NN classifier on cluster centers
    nn_classifier = NearestNeighbors(n_neighbors=1)
    nn_classifier.fit(cluster_centers)

    # Flatten and standardize the test images
    flattened_test_images = x_test.reshape(x_test.shape[0], -1)
    flattened_test_images_scaled = scaler.transform(flattened_test_images)

    # Find nearest cluster centers for each test image
    distances, indices = nn_classifier.kneighbors(flattened_test_images_scaled)

    # Predict labels based on nearest cluster centers
    predicted_labels = clusters[indices].flatten()

    # Compute accuracy
    accuracy = accuracy_score(y_test, predicted_labels)
    print(f"{name} 1-NN Classification Accuracy: {accuracy}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, KernelPCA
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Set the number of samples per class
n_samples_per_class = 100
n_classes = 10
seed = 42  # Seed for reproducibility

# Initialize lists to hold sampled images and labels
sampled_images = []
sampled_labels = []

# Iterate over each class and sample images
for class_label in range(n_classes):
    # Get all images and labels for the current class
    class_images = x_train[y_train == class_label]
    class_labels = y_train[y_train == class_label]

    # Set the seed for numpy's random number generator before sampling
    np.random.seed(seed + class_label)  # Each class with a unique seed

    # Randomly sample n_samples_per_class images from the current class
    sampled_indices = np.random.choice(len(class_images), n_samples_per_class, replace=False)
    sampled_images.append(class_images[sampled_indices])
    sampled_labels.append(class_labels[sampled_indices])

# Convert the lists to numpy arrays
sampled_images = np.concatenate(sampled_images, axis=0)
sampled_labels = np.concatenate(sampled_labels, axis=0)

# Shuffle the sampled dataset
np.random.seed(seed)
shuffled_indices = np.random.permutation(len(sampled_labels))
sampled_images = sampled_images[shuffled_indices]
sampled_labels = sampled_labels[shuffled_indices]

# Flatten the images for clustering
n_samples, width, height = sampled_images.shape
flattened_images = sampled_images.reshape(n_samples, width * height)

# Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
flattened_images_scaled = scaler.fit_transform(flattened_images)

# Run PCA
pca = PCA(n_components=100)
pca.fit(flattened_images_scaled)
pca_images = pca.transform(flattened_images_scaled)

# Run Kernel PCA
kernel_pca = KernelPCA(n_components=100, kernel='rbf', fit_inverse_transform=True, gamma=10)
kernel_pca.fit(flattened_images_scaled)
kernel_pca_images = kernel_pca.transform(flattened_images_scaled)

# Plot the mean image and the first 10 eigenvectors (as images)
mean_image = pca.mean_.reshape(28, 28)
eigenvectors = pca.components_[:10].reshape(10, 28, 28)

plt.figure(figsize=(10, 4))
plt.subplot(2, 6, 1)
plt.imshow(mean_image, cmap='gray')
plt.title('Mean Image')
plt.axis('off')
for i in range(10):
    plt.subplot(2, 6, i + 2)
    plt.imshow(eigenvectors[i], cmap='gray')
    plt.title(f'Eigenvector {i+1}')
    plt.axis('off')

plt.show()

# Plot the eigenvalues (in decreasing order) as a function of dimension
eigenvalues = pca.explained_variance_

plt.figure(figsize=(8, 4))
plt.plot(np.arange(1, 101), eigenvalues, 'o-')
plt.title('Eigenvalues in decreasing order')
plt.xlabel('Dimension')
plt.ylabel('Eigenvalue')
plt.show()


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Set the number of samples per class
n_samples_per_class = 100
n_classes = 10
seed = 42  # 시드 값을 설정합니다.

# Initialize lists to hold sampled images and labels
sampled_images = []
sampled_labels = []

# Iterate over each class and sample images
for class_label in range(n_classes):
    # Get all images and labels for the current class
    class_images = x_train[y_train == class_label]
    class_labels = y_train[y_train == class_label]

    # Set the seed for numpy's random number generator before sampling
    np.random.seed(seed + class_label)  # 각 클래스별로 고유한 시드를 설정합니다.

    # Randomly sample n_samples_per_class images from the current class
    sampled_indices = np.random.choice(len(class_images), n_samples_per_class, replace=False)
    sampled_images.append(class_images[sampled_indices])
    sampled_labels.append(class_labels[sampled_indices])

# Convert the lists to numpy arrays
sampled_images = np.concatenate(sampled_images, axis=0)
sampled_labels = np.concatenate(sampled_labels, axis=0)

# Optionally, shuffle the sampled dataset
np.random.seed(seed)  # 시드를 다시 설정하여 동일한 순서로 섞이도록 합니다.
shuffled_indices = np.random.permutation(len(sampled_labels))
sampled_images = sampled_images[shuffled_indices]
sampled_labels = sampled_labels[shuffled_indices]

# Flatten the images for further processing
n_samples, img_rows, img_cols = sampled_images.shape
flattened_images = sampled_images.reshape(n_samples, img_rows * img_cols)

# Normalize the data
scaler = StandardScaler()
flattened_images = scaler.fit_transform(flattened_images)

# Perform PCA
pca = PCA(n_components=10, random_state=seed)
pca_features = pca.fit_transform(flattened_images)

# Perform Kernel PCA
kpca = KernelPCA(n_components=10, kernel='rbf', gamma=0.1, random_state=seed)
kpca_features = kpca.fit_transform(flattened_images)

# Perform K-means clustering on PCA-reduced features
kmeans_pca = KMeans(n_clusters=n_classes, random_state=seed)
kmeans_pca_labels = kmeans_pca.fit_predict(pca_features)

# Perform K-means clustering on Kernel PCA-reduced features
kmeans_kpca = KMeans(n_clusters=n_classes, random_state=seed)
kmeans_kpca_labels = kmeans_kpca.fit_predict(kpca_features)

# Calculate and print Rand Index and Mutual Information based score for PCA
print("K-means Clustering on PCA-reduced features:")
print("Adjusted Rand Index:", adjusted_rand_score(sampled_labels, kmeans_pca_labels))
print("Normalized Mutual Information Score:", normalized_mutual_info_score(sampled_labels, kmeans_pca_labels))

# Calculate and print Rand Index and Mutual Information based score for Kernel PCA
print("\nK-means Clustering on Kernel PCA-reduced features:")
print("Adjusted Rand Index:", adjusted_rand_score(sampled_labels, kmeans_kpca_labels))
print("Normalized Mutual Information Score:", normalized_mutual_info_score(sampled_labels, kmeans_kpca_labels))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

# Load the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target.astype(int)

# Sample 100 images per class
sampled_indices = []
for class_label in range(10):
    class_indices = np.where(y == class_label)[0]
    sampled_indices.extend(np.random.choice(class_indices, 100, replace=False))

X_sampled = X.iloc[sampled_indices]
y_sampled = y.iloc[sampled_indices]

# Split the sampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42, stratify=y_sampled)

# Fit the 1-NN classifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Define the plot_images function
def plot_images(X_data, y_true, y_pred, indices, title, is_incorrect=False):
    plt.figure(figsize=(10, 2.5))
    for i, idx in enumerate(indices[:3]):
        plt.subplot(1, 3, i+1)
        img = X_data.iloc[idx].values.reshape(28, 28) if hasattr(X_data, 'iloc') else X_data[idx].reshape(28, 28)
        true_label = y_true.iloc[idx] if hasattr(y_true, 'iloc') else y_true[idx]
        pred_label = y_pred.iloc[idx] if hasattr(y_pred, 'iloc') else y_pred[idx]
        plt.imshow(img, cmap='gray')
        plt.title(f"True: {true_label}\nPred: {pred_label}" if is_incorrect else f"Class: {true_label}")
        plt.axis('off')
    plt.suptitle(title, y=1.05)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Identify correct and incorrect predictions
correct_indices = np.where(y_test == y_pred)[0]
incorrect_indices = np.where(y_test != y_pred)[0]

# Plot images for each class
for class_label in np.unique(y_test):
    class_correct = [idx for idx in correct_indices if (y_test.iloc[idx] if hasattr(y_test, 'iloc') else y_test[idx]) == class_label]
    class_incorrect = [idx for idx in incorrect_indices if (y_test.iloc[idx] if hasattr(y_test, 'iloc') else y_test[idx]) == class_label]

    if len(class_correct) > 0:
        plot_images(X_test, y_test, y_pred, class_correct, f"Correctly classified as {class_label}")

    if len(class_incorrect) > 0:
        plot_images(X_test, y_test, y_pred, class_incorrect, f"Incorrectly classified as {class_label}", is_incorrect=True)

# Describe findings
print("Correctly classified images for each class show that the 1-NN classifier can accurately identify many digits.")
print("Incorrectly classified images reveal which digits are commonly misclassified and provide insights into potential areas for model improvement.")
